<font size =8> Barnacle Counting using a Pre-trained model

<font size=4>

In this experiment, I will utilize an API for a YOLO model that has been fine-tuned on a dataset of barnacle images. 


<font size = 5> Importing necessary modules

In [25]:
import tempfile
from inference_sdk import InferenceHTTPClient, InferenceConfiguration
from helpers import *

<font size=6> Image Preprocessing </font>

<font size=4> During initial testing, I observed that the model performs best when the image contains a relatively small number of target objects (barnacles), likely due to the dataset it was trained on.

To optimize detection performance, the first step in this pipeline is to divide each input image into smaller tiles that are stored in a temporary directory within the Notebook. These tiles are then processed sequentially by the model. For each tile, the model detects and counts the barnacles present. Finally, the counts from all tiles are summed to estimate the total number of barnacles in the original image.



In [21]:
with tempfile.TemporaryDirectory() as tmpdir:
    output_directory  = tmpdir
image_path = "C:/Users/arses/Downloads/sample_image.png"

try : 
    os.mkdir(output_directory)
    print(f"Directory '{output_directory}' created successfully.")

#Already created this file
except FileExistsError: 
    print(f"Directory '{output_directory}' already exists.")

#Check access and permissions settings 
except PermissionError: 
    print(f"Permission denied: Unable to create '{output_directory}.")

#Any other type of errors 
except Exception as e: 
    print(f"An error occured:{e}")

Directory 'C:\Users\arses\AppData\Local\Temp\tmp5pey385x' created successfully.


<FONT SIZE =5> Cropping the image into 30 smaller tiles 

In [22]:
crop_image_into_tiles(image_path,output_directory)

Image cropped successfully


<font size = 5> Iniializing the model

In [23]:
custom_configuration= InferenceConfiguration(confidence_threshold=0.3)
CLIENT= InferenceHTTPClient(
    api_url ="https://serverless.roboflow.com", 
    api_key= "CW6dMrLkiMDw9IRcbujY"
)

<font size= 5> Running Loop

In [19]:
number_of_barnacles= 0
number_of_images= directory_size(output_directory)
for i in range(number_of_images):
    with CLIENT.use_configuration(custom_configuration):
        result = CLIENT.infer(f"{output_directory}/tile_{i}.png", model_id = "barnacles-lnd34/1")
    number_of_barnacles+= len(result['predictions'])

print(f"The number of barnacles present in the image is: {number_of_barnacles}")

Number of files in 'C:\Users\arses\AppData\Local\Temp\tmpexrgofj6': 30
The number of barnacles present in the image is: 927


<font size =5> Cleaning the Temporary directory

In [20]:
remove_files_from_directory(output_directory)